In [1]:
import numpy as np
from pathlib import Path
rosbag_path = Path("../rosbag_recordings/20230516_215854_greenbox_posed_2.npz").resolve()

data = np.load(rosbag_path)
rgb_images = data["rgb_image"]
depth_images = data["depth_image"]
pred_masks = []

#T = np.load("/rl_benchmark/real_robot/rosbag_recordings/Tb_b2c.npy")

render_output_file = rosbag_path.parent / f"{rosbag_path.stem}_masks.mp4"
print(f"Saving to {render_output_file}")

import imageio
out_video = imageio.get_writer(str(render_output_file), fps=5, quality=5, macro_block_size=4)

from grounded_sam_track import GroundedSAMTrack
grounded_sam_track = GroundedSAMTrack(predict_gap=9999)

from tqdm import tqdm
for frame_i, (rgb_image, depth_image) in enumerate(zip(tqdm(rgb_images), depth_images)):
    ret = grounded_sam_track.predict_and_track_batch(
        [rgb_image], [frame_i], ["red cube", "green bowl"]
    )
    pred_mask = ret["pred_masks"][0]
    pred_masks.append(pred_mask)
    
    # Save Video
    from seg_and_track_anything.seg_track_anything import draw_mask, colorize_mask
    from mani_skill2.utils.visualization.misc import tile_images

    masked_images = []
    masked_images.append(draw_mask(rgb_image, pred_mask))
    masked_images.append(colorize_mask(pred_mask))
    out_video.append_data(tile_images(masked_images))
out_video.close()

Saving to /rl_benchmark/real_robot/rosbag_recordings/20230516_215854_greenbox_posed_2_masks.mp4


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


_IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight'])

Loading GroundingDINO: Took 3.606 seconds

Loading SAM: Took 5.743 seconds


  0%|          | 0/2162 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:866: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
2023-05-19 00:46:55.480102: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
  0%|          | 1/2162 [00:04<2:29:50,  4.16s/it]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|██████████| 2162/2162 [02:42<00:00, 13.34it/s]


In [2]:
import numpy as np
import cv2
import open3d as o3d
from real_robot.utils.camera import depth2xyz, transform_points

from pyrl.utils.lib3d import np2pcd

def _process_pts(
        pts_lst,
        voxel_downsample_size, nb_neighbors, std_ratio
    ):
        if isinstance(pts_lst, np.ndarray):
            pts_lst = [pts_lst]

        ret_pts_lst = []
        for pts in pts_lst:
            pcd = np2pcd(pts)
            if voxel_downsample_size is not None:
                pcd = pcd.voxel_down_sample(voxel_size=voxel_downsample_size)
            pcd_filter, inlier_inds = pcd.remove_statistical_outlier(
                nb_neighbors=nb_neighbors, std_ratio=std_ratio
            )
            ret_pts_lst.append(np.asarray(pcd_filter.points))

        if len(ret_pts_lst) == 1:
            return ret_pts_lst[0]

        return ret_pts_lst

camera_pose = np.load("/rl_benchmark/real_robot/rosbag_recordings/Tb_b2c.npy")

rgb_image = rgb_images[0]
depth_image = depth_images[0]
pred_mask = pred_masks[0]
xyz_image = depth2xyz(depth_image, *data["intrinsics"])
world_xyz_image = transform_points(xyz_image, camera_pose)

cv2.namedWindow("Color / Depth")
cv2.imshow("Color / Depth", rgb_image)
cv2.waitKey(1)

pcd_vis = o3d.visualization.Visualizer()
pcd_vis.create_window("Point Cloud", width=1280, height=720)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(world_xyz_image.reshape(-1, 3))
pcd.colors = o3d.utility.Vector3dVector(rgb_image.reshape(-1, 3) / 255.0)
coord_frame = o3d.geometry.TriangleMesh().create_coordinate_frame()
pcd_vis.add_geometry(coord_frame)
pcd_vis.add_geometry(pcd)

cube_pts = world_xyz_image[pred_mask == 1]
cube_pts = _process_pts(
    cube_pts, 0.005, 20, 0.005
)
bowl_pts = world_xyz_image[pred_mask == 2]
bowl_pts = _process_pts(
    bowl_pts, 0.005, 20, 0.005
)
cube_pos = np.mean(cube_pts, axis=0)
bowl_pos = np.mean(bowl_pts, axis=0)
# Extract bbox from object_pts
bowl_mins, bowl_maxs = bowl_pts.min(0), bowl_pts.max(0)
cube_mins, cube_maxs = cube_pts.min(0), cube_pts.max(0)

cube_bbox_pos = np.mean([cube_mins, cube_maxs], axis=0)
bowl_bbox_pos = np.mean([bowl_mins, bowl_maxs], axis=0)

pcd_center = o3d.geometry.PointCloud()
pcd_center.points = o3d.utility.Vector3dVector(np.stack([cube_pos, bowl_pos, cube_bbox_pos, bowl_bbox_pos], axis=0))
pcd_center.colors = o3d.utility.Vector3dVector(np.stack([[0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0.]], axis=0))
pcd_vis.add_geometry(pcd_center)

cube_aabb = np2pcd(cube_pts).get_axis_aligned_bounding_box()
bowl_aabb = np2pcd(bowl_pts).get_axis_aligned_bounding_box()
pcd_vis.add_geometry(cube_aabb)
pcd_vis.add_geometry(bowl_aabb)

for i in range(1, len(rgb_images)):
    rgb_image = rgb_images[i]
    depth_image = depth_images[i]
    pred_mask = pred_masks[i]

    xyz_image = depth2xyz(depth_image, *data["intrinsics"])
    world_xyz_image = transform_points(xyz_image, camera_pose)
    pcd.points = o3d.utility.Vector3dVector(world_xyz_image.reshape(-1, 3))
    pcd.colors = o3d.utility.Vector3dVector(rgb_image.reshape(-1, 3) / 255.0)
    

    cube_pts = world_xyz_image[pred_mask == 1]
    cube_pts = _process_pts(
        cube_pts, 0.005, 20, 0.005
    )
    bowl_pts = world_xyz_image[pred_mask == 2]
    bowl_pts = _process_pts(
        bowl_pts, 0.005, 20, 0.005
    )
    cube_pos = np.mean(cube_pts, axis=0)
    bowl_pos = np.mean(bowl_pts, axis=0)
    # Extract bbox from object_pts
    bowl_mins, bowl_maxs = bowl_pts.min(0), bowl_pts.max(0)
    cube_mins, cube_maxs = cube_pts.min(0), cube_pts.max(0)

    cube_bbox_pos = np.mean([cube_mins, cube_maxs], axis=0)
    bowl_bbox_pos = np.mean([bowl_mins, bowl_maxs], axis=0)

    pcd_center.points = o3d.utility.Vector3dVector(np.stack([cube_pos, bowl_pos, cube_bbox_pos, bowl_bbox_pos], axis=0))
    pcd_center.colors = o3d.utility.Vector3dVector(np.stack([[0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0.]], axis=0))
    pcd_vis.update_geometry(pcd_center)

    cube_aabb_new = np2pcd(cube_pts).get_axis_aligned_bounding_box()
    bowl_aabb_new = np2pcd(bowl_pts).get_axis_aligned_bounding_box()
    cube_aabb.max_bound = cube_aabb_new.max_bound
    cube_aabb.min_bound = cube_aabb_new.min_bound
    bowl_aabb.max_bound = bowl_aabb_new.max_bound
    bowl_aabb.min_bound = bowl_aabb_new.min_bound
    pcd_vis.update_geometry(cube_aabb)
    pcd_vis.update_geometry(bowl_aabb)

    pcd_vis.update_geometry(pcd)
    pcd_vis.poll_events()
    pcd_vis.update_renderer()

    depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
    cv2.imshow("Color / Depth", np.hstack([cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR), depth_colormap]))
    if cv2.waitKey(1) == 27:  # ESC
        break

cv2.destroyAllWindows()
pcd_vis.destroy_window()

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
from PIL import Image
Image.fromarray(tile_images(masked_images))